In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 30.6 MB/s 
     |████████████████████████████████| 596 kB 54.4 MB/s 
     |████████████████████████████████| 6.6 MB 64.3 MB/s 
     |████████████████████████████████| 101 kB 4.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from random import choice

import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import os

In [ ]:
folklore = pd.read_csv('08-folklore_deluxe_version.csv')
evermore = pd.read_csv('09-evermore_deluxe_version.csv')
lover = pd.read_csv('07-lover.csv')
reputation = pd.read_csv('06-reputation.csv')
the1989 = pd.read_csv('05-1989_deluxe.csv')
red = pd.read_csv('04-red_deluxe_edition.csv')
speak = pd.read_csv('03-speak_now_deluxe_package.csv')
fearless = pd.read_csv('02-fearless_taylors_version.csv')
taylor = pd.read_csv('01-taylor_swift.csv')

In [ ]:
df = pd.concat([folklore, evermore, lover,reputation,the1989,red,speak,fearless,taylor], axis=0)
df.head()

,album_name,track_title,track_n,lyric,line
0,folklore (deluxe version),the 1,1,"I'm doing good, I'm on some new shit",1
1,folklore (deluxe version),the 1,1,"Been saying ""yes"" instead of ""no""",2
2,folklore (deluxe version),the 1,1,"I thought I saw you at the bus stop, I didn't ...",3
3,folklore (deluxe version),the 1,1,I hit the ground running each night,4
4,folklore (deluxe version),the 1,1,I hit the Sunday matinée,5


In [ ]:
tok = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
df = df.reset_index()

test_set = df.sample(n = 200)
df = df.loc[~df.index.isin(test_set.index)]

#Reset the indexes
test_set = test_set.reset_index()
df = df.reset_index()

In [ ]:
from torch.utils.data import Dataset, DataLoader

class SongLyrics(Dataset):  
    def __init__(self, control_code, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in df['lyric']:
          self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))
          self.lyrics_count = len(self.lyrics)
                         
    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]
dataset = SongLyrics(df['lyric'], gpt2_type="gpt2")   

In [ ]:
#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [ ]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=5, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [ ]:
model = train(dataset, model, tok)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Training epoch 0
0


8158it [09:31, 14.26it/s]


Training epoch 1
tensor(0.0938, device='cuda:0', grad_fn=<NllLossBackward0>)


8158it [09:38, 14.11it/s]


...

8158it [09:36, 14.14it/s]


Training epoch 3
tensor(0.0949, device='cuda:0', grad_fn=<NllLossBackward0>)


8158it [09:37, 14.14it/s]


Training epoch 4
tensor(0.1397, device='cuda:0', grad_fn=<NllLossBackward0>)


8158it [09:33, 14.22it/s]


In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_lyrics = []
  for i in range(len(test_data)):
    x = generate(model.to('cpu'), tok, test_data['lyric'][i], entry_count=1)
    generated_lyrics.append(x)
  return generated_lyrics

#Run the functions to generate the lyrics
generated_lyrics = text_generation(test_set)

100%|██████████| 1/1 [00:04<00:00,  4.58s/it]


In [ ]:
generated_lyrics

[["I'll be right here on the ground with you, even if it means going back to your side of the tunnel to the other side of the room. I'm going to try to take<|endoftext|>"],
 ["And when we had that fight out in the rain, I went out to the river with the other monks and prayed, 'O Lord, we are all together, we will all fight' (D<|endoftext|>"],
 ["But you changed the rules everyday.\n\nAnd now, I'm happy to say you're not alone.\n\nWe all just need to make a change.<|endoftext|>"],
 ['If you say you\'d rather love than fight, what do you say to me?"<|endoftext|>'],
 ['If I didn\'t know better, it would have been for me.\n\n"He was one of those things that I could have lost, just like he was that I didn<|endoftext|>'],
 ['You never let me drive."<|endoftext|>'],
 ['Sorry that I hurt you guys, I\'m just going to play harder. I\'m playing with myself."<|endoftext|>'],
 ['Oh, oh-oh, oh-oh-oh-oh-oh-oh!", "Oh, oh, oh-oh!", "Oh-oh!", "Oh!", "Oh<|endoftext|>'],
 ['I was begging you, "Please don\